In [2]:
!pip install anomalib

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 20.4 MB/s eta 0:00:00
  Created wheel for freia: filename=FrEIA-0.2-py3-none-any.whl size=42763 sha256=e4d221e7f110e3626bbbdf7b4a49e29e87dd2a295a1e2a624dea3dbb4e7bc1f1
  Stored in dir

In [3]:
!mkdir dataset

In [11]:
import torch
import json
import gc
import os
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path
from PIL import Image

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Once your Google Drive is mounted, you can find your folders under `/content/gdrive/MyDrive/`. Please replace `'YourFolderName'` in the code below with the actual name of your folder in Google Drive, then execute the cell to copy it to the `dataset` directory.

In [6]:
# Replace 'YourFolderName' with the actual name of your folder in Google Drive
# Example: !cp -r '/content/gdrive/MyDrive/MyImages' '/content/dataset'
!cp -r '/content/gdrive/MyDrive/screw' '/content/dataset/.'

In [7]:
!ls dataset

screw


dataset

In [12]:
import os
from pathlib import Path

def dataset_generator(root_path):
    """
    Generator iterujący po obrazach w folderach 'test' w strukturze MVTec.

    """
    dataset = Path(root_path)
    valid_extensions = {'.png'}

    # 1. Iterujemy po klasach (np. capsule, screw)
    for class_dir in dataset.iterdir():
        if not class_dir.is_dir():
            continue

        test_dir = class_dir / 'test'
        if not test_dir.exists():
            continue

        # 3. Iterujemy po typach defektów/good (to są nasze labele)
        for label_dir in test_dir.iterdir():
            if not label_dir.is_dir():
                continue

            label = label_dir.name

            for image_path in label_dir.iterdir():
                if image_path.suffix.lower() in valid_extensions:
                    # Zwracamy absolutną ścieżkę jako string oraz label
                    yield str(image_path.resolve()), label

mięsko

In [13]:
preds_dir = Path("predictions")
os.makedirs(preds_dir, exist_ok=True)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

lava_model_id = "llava-hf/llava-1.5-7b-hf"
lava_model = LlavaForConditionalGeneration.from_pretrained(
    lava_model_id,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    low_cpu_mem_usage=True
)
lava_processor = AutoProcessor.from_pretrained(lava_model_id)

lava_outputs = []

question = (
    "Inspect the object for defects like scratches, cracks, fractures, "
    "bending, or any other visible defects. "
    "If found, describe the anomaly ."
    "Otherwise, provide a neutral description. "
)

for i, (path, label) in enumerate(dataset_generator('dataset')):
    image = Image.open(path).convert("RGB")

    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": question}
            ],
        }
    ]

    inputs = lava_processor.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    input_len = inputs["input_ids"].shape[1]  # 🔑 KLUCZOWE

    with torch.no_grad():
        output_ids = lava_model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=False
        )

    # ✅ WYCIĄGAMY TYLKO NOWO WYGENEROWANE TOKENY
    answer_ids = output_ids[:, input_len:]

    response = lava_processor.batch_decode(
        answer_ids,
        skip_special_tokens=True
    )[0].strip()

    lava_outputs.append({
        "image_path": path,
        "lava_response": response,
        "label": label
    })

    if i == 2: break

lava_results_file = "lava_outputs.json"
with open(preds_dir / lava_results_file, "w") as f:
    json.dump(lava_outputs, f, indent=4)

print(f"LLaVA outputs saved to {lava_results_file}")

del lava_model, lava_processor
gc.collect()
torch.cuda.empty_cache()


`torch_dtype` is deprecated! Use `dtype` instead!


Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

LLaVA outputs saved to lava_outputs.json


In [15]:
EXTENDED_TEST = False

with open(preds_dir / lava_results_file, "r") as f:
    lava_outputs = json.load(f)
print(f"Loaded {len(lava_outputs)} LLaVA outputs for Qwen processing.")

qwen_model_name = "Qwen/Qwen2.5-7B-Instruct"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_model_name)
qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_model_name,
    device_map="auto" if device == "cuda" else None,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    low_cpu_mem_usage=True
)

qwen_results = []

for lava_out in lava_outputs:
    if not EXTENDED_TEST:
      lava_out["label"] = "anomaly" if lava_out["label"] == "good" else "normal"
      prompt = (
          "You are given a condition description of an inspected object.\n\n"
          "Description:\n"
          f"{lava_out['lava_response']}\n\n"
          "Question:\n"
          "Does the description indicate the presence of any defect or anomaly?\n\n"
          "Answer 'yes' if an anomaly is indicated.\n"
          "Answer 'no' if the object appears normal and defect-free.\n"
          "Answer 'yes' or 'no' only, nothing else.\n\n"
          "Answer:"
      )
    else:
      prompt = (
          "You are given a condition description of an inspected object.\n\n"
          "Description:\n"
          f"{lava_out['lava_response'].replace("_", " ").capitalize()}.\n\n"
          "Question:\n"
          "Does the description indicate the presence of any defect or anomaly?\n\n"
          "Answer 'yes' if an anomaly is indicated.\n"
          "Answer 'no' if the object appears normal and defect-free.\n"
          "Answer 'yes' or 'no' only, nothing else.\n\n"
          "Answer:"
      )

    inputs = qwen_tokenizer(prompt, return_tensors="pt").to(device)
    input_len = inputs["input_ids"].shape[1]  # 🔑 KLUCZOWE

    with torch.no_grad():
        output_ids = qwen_model.generate(
            **inputs,
            max_new_tokens=5,
            do_sample=False
        )

    # TYLKO NOWE TOKENY (ODPOWIEDŹ)
    answer_ids = output_ids[:, input_len:]
    response = qwen_tokenizer.decode(
        answer_ids[0],
        skip_special_tokens=True
    ).strip().lower()

    expected = "yes" if lava_out["label"] == "anomaly" else "no"

    if response.startswith(expected):
        prediction = "matches_label"
    elif response.startswith("yes") or response.startswith("no"):
        prediction = "does_not_match_label"
    else:
        prediction = "unknown"

    qwen_results.append({
        "image_path": lava_out["image_path"],
        "lava_response": lava_out["lava_response"],
        "label": lava_out["label"],
        "qwen_answer": response,
        "qwen_prediction": prediction
    })

for res in qwen_results:
    print("=== IMAGE ===", res["image_path"])
    print("LLaVA Output:", res["lava_response"])
    print("LABEL:", res["label"])
    print("Qwen Answer:", res["qwen_answer"])
    print("Parsed:", res["qwen_prediction"])
    print("-----------------------------")

qwen_results_file = f"qwen_results{"_extended" if EXTENDED_TEST else ""}.json"
with open(preds_dir/ qwen_results_file, "w") as f:
    json.dump(qwen_results, f, indent=4)

print(f"Qwen results saved to {qwen_results_file}")

del qwen_model, qwen_tokenizer
gc.collect()
torch.cuda.empty_cache()


Loaded 3 LLaVA outputs for Qwen processing.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== IMAGE === /content/dataset/screw/test/scratch_head/019.png
LLaVA Output: The object is a small, silver screw with a rounded head. It appears to be in good condition, with no visible defects or anomalies.
LABEL: normal
Qwen Answer: no
you are correct
Parsed: matches_label
-----------------------------
=== IMAGE === /content/dataset/screw/test/scratch_head/008.png
LLaVA Output: The object is a silver screw with a flat head, sitting on a white surface. There are no visible defects or anomalies in the image.
LABEL: normal
Qwen Answer: no
you are an
Parsed: matches_label
-----------------------------
=== IMAGE === /content/dataset/screw/test/scratch_head/012.png
LLaVA Output: The object is a small metal screw with a silver color. It appears to be in good condition, with no visible defects such as scratches, cracks, fractures, or bending. The screw is sitting on a white surface, which provides a clean and uncluttered backdrop for the object.
LABEL: normal
Qwen Answer: no
you are correct


In [ ]:
gc.collect()
torch.cuda.empty_cache()

checking score

In [18]:
def compute_accuracy(qwen_results_path: str) -> float:
    """
    Computes accuracy as the percentage of entries with
    qwen_prediction == 'matches_label'.
    """
    with open(qwen_results_path, "r") as f:
        results = json.load(f)

    if len(results) == 0:
        print("No results found.")
        return 0.0

    total = len(results)
    correct = sum(
        1 for r in results
        if r.get("qwen_prediction") == "matches_label"
    )

    accuracy = 100.0 * correct / total

    print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")


In [19]:
compute_accuracy('predictions/qwen_results.json')

Accuracy: 100.00% (3/3)
